In [1]:
pip install pandas numpy matplotlib prophet scikit-learn streamlit fredapi yfinance plotly


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install -r requirements.txt


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install streamlit


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
# 🔧 Manipulación y visualización
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 🔮 Modelado
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error

# 🌐 APIs externas
import yfinance as yf
from fredapi import Fred

# 📊 Streamlit (para la app final)
import streamlit as st


In [9]:
# 🔑 Inicializar FRED
from fredapi import Fred
fred = Fred(api_key='437ffc22620f0fe3615350b1764f112b')

# 📥 Descargar series desde FRED
inflacion_raw = fred.get_series('CPIAUCSL')
tasa_raw = fred.get_series('FEDFUNDS')

# 🧼 Convertir a DataFrame con fechas
inflacion_mejorado = pd.DataFrame({
    'Fecha': inflacion_raw.index,
    'Inflacion': inflacion_raw.values
})
tasa_mejorado = pd.DataFrame({
    'Fecha': tasa_raw.index,
    'Tasa_Fed': tasa_raw.values
})

# 📅 Filtrar rango deseado
inflacion_mejorado = inflacion_mejorado[(inflacion_mejorado['Fecha'] >= '2010-01-01') & (inflacion_mejorado['Fecha'] <= '2025-12-31')]
tasa_mejorado = tasa_mejorado[(tasa_mejorado['Fecha'] >= '2010-01-01') & (tasa_mejorado['Fecha'] <= '2025-12-31')]


In [13]:
# 📥 Descargar series desde Yahoo Finance
usdeur_raw = yf.download('EURUSD=X', start='2010-01-01', end='2025-12-31')
dxy_raw = yf.download('DX-Y.NYB', start='2010-01-01', end='2025-12-31')

# 🧼 Limpiar y renombrar
usdeur_mejorado = usdeur_raw[['Close']].reset_index().rename(columns={'Date': 'Fecha', 'Close': 'USD_EUR'})
dxy_mejorado = dxy_raw[['Close']].reset_index().rename(columns={'Date': 'Fecha', 'Close': 'DXY'})

# 🧼 Eliminar MultiIndex si existe
usdeur_mejorado.columns = usdeur_mejorado.columns.get_level_values(0)
dxy_mejorado.columns = dxy_mejorado.columns.get_level_values(0)

# 📅 Convertir fechas a mensual y ordenar
usdeur_mejorado = usdeur_mejorado.set_index('Fecha').resample('MS').mean().reset_index().sort_values('Fecha')
dxy_mejorado = dxy_mejorado.set_index('Fecha').resample('MS').mean().reset_index().sort_values('Fecha')
inflacion_mejorado = inflacion_mejorado.sort_values('Fecha')
tasa_mejorado = tasa_mejorado.sort_values('Fecha')

# 🧩 Unir todas las series
df_mejorado = usdeur_mejorado.merge(dxy_mejorado, on='Fecha', how='inner')
df_mejorado = df_mejorado.merge(inflacion_mejorado, on='Fecha', how='inner')
df_mejorado = df_mejorado.merge(tasa_mejorado, on='Fecha', how='inner')

# 🧽 Eliminar nulos y guardar
df_mejorado = df_mejorado.dropna()
df_mejorado.to_csv('dataset_mejorado.csv', index=False)


C:\Users\cpuertas\AppData\Local\Temp\ipykernel_16364\2935753380.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  usdeur_raw = yf.download('EURUSD=X', start='2010-01-01', end='2025-12-31')
[*********************100%***********************]  1 of 1 completed
C:\Users\cpuertas\AppData\Local\Temp\ipykernel_16364\2935753380.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  dxy_raw = yf.download('DX-Y.NYB', start='2010-01-01', end='2025-12-31')
[*********************100%***********************]  1 of 1 completed


In [15]:
# 📥 Cargar dataset
df = pd.read_csv('dataset_mejorado.csv')

# 🧼 Renombrar columnas para Prophet
df = df.rename(columns={
    'Fecha': 'ds',
    'USD_EUR': 'y',
    'DXY': 'dxy_mejorado',
    'Inflacion': 'inflacion_mejorado',
    'Tasa_Fed': 'tasa_mejorado'
})

# 🧽 Convertir tipos
df['ds'] = pd.to_datetime(df['ds'])
df = df.dropna()

# 🔮 Entrenar modelo Prophet con regresores
modelo = Prophet(yearly_seasonality=False)
modelo.add_regressor('dxy_mejorado')
modelo.add_regressor('inflacion_mejorado')
modelo.add_regressor('tasa_mejorado')
modelo.fit(df)

# 📈 Crear proyección futura (36 meses)
futuro = modelo.make_future_dataframe(periods=36, freq='MS')
ultimo = df.iloc[-1]

# Trayectorias suavizadas
futuro['dxy_mejorado'] = np.linspace(ultimo['dxy_mejorado'], ultimo['dxy_mejorado'] * 0.98, len(futuro))
futuro['inflacion_mejorado'] = np.linspace(ultimo['inflacion_mejorado'], ultimo['inflacion_mejorado'] * 0.97, len(futuro))
futuro['tasa_mejorado'] = np.linspace(ultimo['tasa_mejorado'], ultimo['tasa_mejorado'] * 0.96, len(futuro))

# 🔮 Generar predicción
forecast = modelo.predict(futuro)

# 📊 Validación del modelo (solo sobre datos históricos)
real = df['y'].reset_index(drop=True)
pred = forecast.loc[:len(real)-1, 'yhat'].reset_index(drop=True)

mae = mean_absolute_error(real, pred)
rmse = mean_squared_error(real, pred) ** 0.5


print(f"✅ MAE Prophet mejorado: {mae:.4f}")
print(f"✅ RMSE Prophet mejorado: {rmse:.4f}")

# 💾 Guardar modelo
import pickle
with open('modelo_prophet_mejorado.pkl', 'wb') as f:
    pickle.dump(modelo, f)


11:56:33 - cmdstanpy - INFO - Chain [1] start processing
11:56:33 - cmdstanpy - INFO - Chain [1] done processing


✅ MAE Prophet mejorado: 0.1063
✅ RMSE Prophet mejorado: 0.1201


In [16]:
import pandas as pd
import yfinance as yf
from fredapi import Fred
from datetime import datetime

# Clave API de FRED
fred = Fred(api_key="437ffc22620f0fe3615350b1764f112b")

# 1. Descargar USD/EUR y DXY desde Yahoo Finance
eurusd = yf.download("EURUSD=X", start="2010-01-01", end=datetime.today().strftime('%Y-%m-%d'))
dxy = yf.download("DX-Y.NYB", start="2010-01-01", end=datetime.today().strftime('%Y-%m-%d'))

# 2. Descargar inflación y tasa de interés desde FRED
inflacion = fred.get_series("CPIAUCSL")
tasa_fed = fred.get_series("FEDFUNDS")

# 3. Formatear y unir los datos
df = pd.DataFrame()
df["Fecha"] = eurusd.index
df["USD_EUR"] = eurusd["Close"].values
df["DXY"] = dxy["Close"].reindex(eurusd.index).values
df["Inflacion"] = inflacion.reindex(eurusd.index.to_period("M").to_timestamp()).values
df["Tasa_Fed"] = tasa_fed.reindex(eurusd.index.to_period("M").to_timestamp()).values

# 4. Guardar el dataset actualizado
df.dropna(inplace=True)
df.to_csv("dataset_final_economico.csv", index=False)


C:\Users\cpuertas\AppData\Local\Temp\ipykernel_16364\4016925914.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  eurusd = yf.download("EURUSD=X", start="2010-01-01", end=datetime.today().strftime('%Y-%m-%d'))
[*********************100%***********************]  1 of 1 completed
C:\Users\cpuertas\AppData\Local\Temp\ipykernel_16364\4016925914.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  dxy = yf.download("DX-Y.NYB", start="2010-01-01", end=datetime.today().strftime('%Y-%m-%d'))
[*********************100%***********************]  1 of 1 completed


In [17]:
from prophet import Prophet
import pandas as pd
import numpy as np

# Cargar el dataset actualizado
df = pd.read_csv("dataset_final_economico.csv")

# Preparar los datos para Prophet
df_prophet = df[["Fecha", "USD_EUR"]].rename(columns={"Fecha": "ds", "USD_EUR": "y"})

# Entrenar el modelo
modelo = Prophet()
modelo.fit(df_prophet)

# Crear horizonte de predicción (3 años)
future = modelo.make_future_dataframe(periods=36, freq="M")
forecast = modelo.predict(future)

# Escenarios
forecast["escenario_neutro"] = forecast["yhat"]
forecast["escenario_positivo"] = forecast["yhat"] * 1.05  # crecimiento del 5%
forecast["escenario_negativo"] = forecast["yhat"] * 0.95  # caída del 5%

# Guardar resultados
forecast_final = forecast[["ds", "escenario_neutro", "escenario_positivo", "escenario_negativo"]]
forecast_final.to_csv("forecast_mejorado.csv", index=False)


12:19:02 - cmdstanpy - INFO - Chain [1] start processing
12:19:03 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\cpuertas\AppData\Roaming\Python\Python313\site-packages\prophet\forecaster.py:1872: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(


In [1]:
pip install plotly


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [21]:
# 📅 Generar fechas futuras (ej. 180 días desde el último dato)
future = modelo_prophet.make_future_dataframe(periods=180)

# 🔮 Generar predicciones
forecast = modelo_prophet.predict(future)

# ✅ Guardar el forecast si quieres reutilizarlo
forecast.to_csv("forecast_mejorado.csv", index=False)

# 🧾 Confirmar columnas disponibles
st.write("Columnas generadas por Prophet:")
st.write(forecast.columns.tolist())


NameError: name 'modelo_prophet' is not defined